In [1]:
import h5py
from obspy import read, Trace, Stream, UTCDateTime
from obspy.core import AttribDict
from obspy.io.segy.segy import SEGYTraceHeader, SEGYBinaryFileHeader
from obspy.io.segy.core import _read_segy
import numpy as np
import sys

In [2]:
# Set the very simple GUI environment 
from tkinter import filedialog as fd
input_files = fd.askopenfilenames()

In [3]:
# Define the input and output file paths 
#input_file="0000653260_2023-02-06_01.24.58.96832.hdf5"
#input_file="./230222/0000653259_2023-02-06_01.24.57.46832.hdf5"
#output_file="example_hdf5.segy"

In [4]:

def hdf5_to_segy (data, ntraces, sampling_interval, timeflag, output_file):
    # Create an ObsPy stream object 
    stream= Stream()
    
    
    # Loop over each trace and add it to the stream
    for i in range(ntraces):
        trace = Trace(data=np.require(data[:,i], dtype=np.float32))
        trace.stats.delta = sampling_interval # segy는 0.06 ms가 최대 성긴 샘플링 & 3만 샘플 최대.
        # trace.stats.starttime = UTCDataTime ... file name stamp 이용해야 함. 원노트 obspy segy header 참조
        
        if not hasattr(trace.stats, 'segy.trace_header'):
            trace.stats.segy={}
        trace.stats.segy.trace_header  = SEGYTraceHeader()
        trace.stats.segy.trace_header.trace_sequence_number_within_line= i # SEQ number
        trace.stats.segy.trace_header.original_field_record_number= int(timeflag) # FFID
        #trace.stats.segy.trace_header.shotpoint_number= int(timeflag) # ShotpointNumber?
        trace.stats.segy.trace_header.energy_source_point_number= int(timeflag) # ShotpointNumber
        #trace.stats.segy.trace_header.ensemble_number= int(timeflag) # CMP 
        trace.stats.segy.trace_header.trace_number_within_the_original_field_record = i # Channel number 
        
        if i % 1000 == 0:
            print(i, "/", ntraces, 'appended')
        
        stream.append(trace)
    print("done ... ")
    
    stream.stats=AttribDict()
    stream.stats.textualfile_header = 'Textual Header'
    stream.stats.binary_file_header=SEGYBinaryFileHeader()
    stream.stats.binary_file_header.trace_sorting_code=5
    
    # write out to SEGY file
    stream.write(output_file, format="SEGY", data_encoding=1, byteorder=sys.byteorder) 
    
    return 



In [6]:
for input_file in input_files:
    print(input_file)
    if input_file[-4:] == "hdf5":
        print("Proceeding ...................................")
        # Read in HDF5 file and See the Metadata Structure
        with h5py.File(input_file, "r") as f:
            #data= f['NOISE1'][:] # TEST DATA for Display
            data= f['DAS'][:] 
            ns, ntraces = data.shape
            print("ns, ntraces", "=", ns, ntraces)
            for name in f:
                print("===========================================================================================")
                print(name)
                print("===========================================================================================")
                if name=="DAQ":
                    for item in f[name]:
                        print(item, "=", f[name][item][0])
                        if item == "RepetitionFrequency":
                            sampling_interval = 1./f[name][item][0]
                if name=="Interrogator":
                    for item in f[name]:
                        print(item, "=", f[name][item][0])
                if name=="Metadata":
                    for item in f[name]:
                        print(item, "=", f[name][item][0])
                        if item == 'Timestamp':
                            GT=UTCDateTime(f[name][item][0])
                            timeflag= str("%03d" % GT.julday) + str("%02d" % GT.hour) + str(str("%02d" % GT.minute)) + str(str( "%02d" % GT.second))
                if name=="ProcessingServer":
                    for item in f[name]:
                        print(item, "=", f[name][item][0])

        print("================================================================================================")
        #sampling_interval=2000*60*5/1200*sampling_interval
        #sampling_interval=sampling_interval*1/10 # For SEGY TEST ONLY
        print("sampling_interval = ", sampling_interval )
    else:
        print("Please select a proper file format")
        quit()
        
    # Writing SEGY in two ways ...[ns over 30,000 and else ...]
    limit_ns=30000
    if ns > limit_ns:
        nsplit=int(ns/limit_ns)
        
        isplit = 0
        while isplit < nsplit:
            if isplit == nsplit -1:
                tmpdata_end = data[ (isplit)*limit_ns : ns+1, :] # 파이썬은 [이상:미만] 임에 주의.
                print("isplit=", isplit, np.shape(tmpdata_end))
                output_file=input_file[:-4]+str("%02d" % isplit)+"."+"segy"
                print(output_file)
                hdf5_to_segy (tmpdata, ntraces, sampling_interval, timeflag, output_file)
                print(isplit+1, "/", nsplit, "is done totally")
                print("---------------------------------------------------------------------------")
                
            else:
                tmpdata = data[ (isplit)*limit_ns : (isplit+1)*limit_ns, :]
                print("isplot = ", isplit, np.shape(tmpdata))
                output_file=input_file[:-4]+str("%02d" % isplit)+"."+"segy"
                print(output_file)
                hdf5_to_segy (tmpdata, ntraces, sampling_interval, timeflag, output_file)
                print(isplit+1, "/", nsplit, "is done totally")
                print("---------------------------------------------------------------------------")

            isplit = isplit + 1 
            
    else:
        output_file=input_file[:-4]+"segy"
        print(output_file)
        hdf5_to_segy (data, ntraces, sampling_interval, timeflag, output_file)
        print("1/1", "is done totally")
        print("---------------------------------------------------------------------------")



M:/한전_AP_das/230222/0000653249_2023-02-06_00.59.57.46832.hdf5
Proceeding ...................................
ns, ntraces = 600000 7031
DAQ
ACCoupled = 0
BandwidthLimiterEnabled = 0
BaseCardVersion = b'12.38'
ChannelCount = 2
ClockFrequency = 10
ClockOutputEnabled = 0
Driver = b'Linux 64bit'
DriverVersion = b'5.5.15170'
ExtendedModuleVersion = b'0.0'
ExternalClockEnabled = 1
FiftyOhmEnabled = 1
InputPathEnabled = 1
InstalledMemory = 4096
KernelVersion = b'1.21.14046'
ModuleVersion = b'12.0'
MultipleTriggerEnabled = 1
PositionEnd = 35903.49216614449
PositionStart = 0.0
ProductionDate = b'Week 24 of 2019'
RepetitionFrequency = 2000
SampleEnd = 84400
SampleStart = 0
SamplingInterval = 0.42539683
SamplingRate = 400
SerialNumber = 14004
SpatialPoints = 7031
TriggerACCoupleEnabled = 0
TriggerAnalogModeEnabled = 1
TriggerFiftyOhmEnabled = 1
TriggerLevel = 1500
Type = b'M4i.4480-x8'
VoltageRange = 2500
DAS
Interrogator
ActiveOpticalChannelNumber = 1
BaseBoardRevision = 0
BaseClockPeriod = 5
Clo

1000 / 7031 appended
2000 / 7031 appended
3000 / 7031 appended
4000 / 7031 appended
5000 / 7031 appended
6000 / 7031 appended
7000 / 7031 appended
done ... 
13 / 20 is done totally
---------------------------------------------------------------------------
isplot =  13 (30000, 7031)
M:/한전_AP_das/230222/0000653249_2023-02-06_00.59.57.46832.13.segy
0 / 7031 appended
1000 / 7031 appended
2000 / 7031 appended
3000 / 7031 appended
4000 / 7031 appended
5000 / 7031 appended
6000 / 7031 appended
7000 / 7031 appended
done ... 
14 / 20 is done totally
---------------------------------------------------------------------------
isplot =  14 (30000, 7031)
M:/한전_AP_das/230222/0000653249_2023-02-06_00.59.57.46832.14.segy
0 / 7031 appended
1000 / 7031 appended
2000 / 7031 appended
3000 / 7031 appended
4000 / 7031 appended
5000 / 7031 appended
6000 / 7031 appended
7000 / 7031 appended
done ... 
15 / 20 is done totally
---------------------------------------------------------------------------
isplot = 